<a href="https://colab.research.google.com/github/hinzle/cognizant/blob/main/PySpark_and_SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to this course "Getting started with Apache Spark"

![PySpark](https://drive.google.com/uc?id=1oU2tHXn4Tb4NJ0GQLbFQanLUVWj-3M-G)

## Contents
- Setting up the environment
- Read data
- Preprocessing data using SparkML
- Modeling using SparkML
- Prediction on Test data
- Evaluation of predictions

## Setting up the PySpark environment

- You can use the below cell to install all the required libraries and files

In [ ]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import json
import pyspark.sql.functions as F
import pyspark.sql
from pyspark.sql.functions import col, skewness, kurtosis
from pyspark.context import SparkContext
from pyspark.sql.functions import * 
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import when
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.functions import from_unixtime, unix_timestamp
from pyspark.sql.types import StringType
from datetime import date, timedelta, datetime
import time

### Initialize SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark.version

'3.3.0'

### Read data
Dataset (Wine quality red): https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

In [ ]:
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv -P sample_data/

In [ ]:
# We can set header='true' and inferSchema='true' to infer the schema while reading the data

filepath = "sample_data/winequality-red.csv"
spark_df = spark.read.format('csv').options(header='true', inferSchema='true', delimiter=";").load(filepath)
spark_df.show(5, truncate=False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|pH  |sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|7.4          |0.7             |0.0        |1.9           |0.076    |11.0               |34.0                |0.9978 |3.51|0.56     |9.4    |5      |
|7.8          |0.88            |0.0        |2.6           |0.098    |25.0               |67.0                |0.9968 |3.2 |0.68     |9.8    |5      |
|7.8          |0.76            |0.04       |2.3           |0.092    |15.0               |54.0                |0.997  |3.26|0.65     |9.8    |5      |
|11.2         |0.28            |0.56       |1.9           |0.075    |17.0               |60.0       

## Introduction

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

# StringIndexer is similar to labelencoder which gives a label to each category
# OneHotEncoder created onehot encoding vector
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# VectorAssembler is used to create vector from the features. MOdeling takes vector as an input
from pyspark.ml.feature import VectorAssembler

# DecisionTreeClassifier is used for classiication problems
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
# Create a categorical column for explanation purpose
spark_df = spark_df.withColumn("alcohol", F.when(F.col("alcohol") > 10.5, "High").otherwise("Low"))
spark_df.show(3, truncate=False)

spark_df.groupby("alcohol").count().show(), spark_df.select("quality").distinct().show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|pH  |sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|7.4          |0.7             |0.0        |1.9           |0.076    |11.0               |34.0                |0.9978 |3.51|0.56     |Low    |5      |
|7.8          |0.88            |0.0        |2.6           |0.098    |25.0               |67.0                |0.9968 |3.2 |0.68     |Low    |5      |
|7.8          |0.76            |0.04       |2.3           |0.092    |15.0               |54.0                |0.997  |3.26|0.65     |Low    |5      |
+-------------+----------------+-----------+--------------+---------+-------------------+-----------

(None, None)

In [ ]:
(train_df, test_df) = spark_df.randomSplit([0.8, 0.2], 11)
print("Number of train samples: " + str(train_df.count()))
print("Number of test samples: " + str(test_df.count()))

Number of train samples: 1279
Number of test samples: 320


## Modeling

In [ ]:
train_df.show(3, truncate=False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|pH  |sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|4.6          |0.52            |0.15       |2.1           |0.054    |8.0                |65.0                |0.9934 |3.9 |0.56     |High   |4      |
|4.7          |0.6             |0.17       |2.3           |0.058    |17.0               |106.0               |0.9932 |3.85|0.6      |High   |6      |
|4.9          |0.42            |0.0        |2.1           |0.048    |16.0               |42.0                |0.99154|3.71|0.74     |High   |7      |
+-------------+----------------+-----------+--------------+---------+-------------------+-----------

In [ ]:
alcohol_indexer = StringIndexer(inputCol="alcohol", outputCol="alcoholIndex")
alcohol_indexer = alcohol_indexer.fit(train_df)
train_df = alcohol_indexer.transform(train_df)
train_df.show(3, truncate=False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|pH  |sulphates|alcohol|quality|alcoholIndex|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------+
|4.6          |0.52            |0.15       |2.1           |0.054    |8.0                |65.0                |0.9934 |3.9 |0.56     |High   |4      |1.0         |
|4.7          |0.6             |0.17       |2.3           |0.058    |17.0               |106.0               |0.9932 |3.85|0.6      |High   |6      |1.0         |
|4.9          |0.42            |0.0        |2.1           |0.048    |16.0               |42.0                |0.99154|3.71|0.74     |High   |7      |1.0         |
+-------------+-------

In [ ]:
print(train_df.columns)

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality', 'alcoholIndex']


In [ ]:
inputCols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcoholIndex']

outputCol = "features"
vector_assembler = VectorAssembler(inputCols = inputCols, outputCol = outputCol)
train_df = vector_assembler.transform(train_df)

In [ ]:
train_df.show(3, truncate=False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------+--------------------------------------------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|pH  |sulphates|alcohol|quality|alcoholIndex|features                                                |
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------+--------------------------------------------------------+
|4.6          |0.52            |0.15       |2.1           |0.054    |8.0                |65.0                |0.9934 |3.9 |0.56     |High   |4      |1.0         |[4.6,0.52,0.15,2.1,0.054,8.0,65.0,0.9934,3.9,0.56,1.0]  |
|4.7          |0.6             |0.17       |2.3           |0.058    |17.0               |106.0               |0.9932 |3.

In [ ]:
modeling_df = train_df.select(['features', 'quality'])
modeling_df.show(3, truncate=False)

+--------------------------------------------------------+-------+
|features                                                |quality|
+--------------------------------------------------------+-------+
|[4.6,0.52,0.15,2.1,0.054,8.0,65.0,0.9934,3.9,0.56,1.0]  |4      |
|[4.7,0.6,0.17,2.3,0.058,17.0,106.0,0.9932,3.85,0.6,1.0] |6      |
|[4.9,0.42,0.0,2.1,0.048,16.0,42.0,0.99154,3.71,0.74,1.0]|7      |
+--------------------------------------------------------+-------+
only showing top 3 rows



In [ ]:
# Create DecisionTreeClassifier model
dt_model = DecisionTreeClassifier(labelCol="quality", featuresCol="features")

# Train model with Training Data
dt_model = dt_model.fit(modeling_df)

In [ ]:
predictions = dt_model.transform(modeling_df)
predictions.show(5, truncate=False)

+--------------------------------------------------------+-------+----------------------------------------+----------------------------------------------------------------------------------------------------------------------+----------+
|features                                                |quality|rawPrediction                           |probability                                                                                                           |prediction|
+--------------------------------------------------------+-------+----------------------------------------+----------------------------------------------------------------------------------------------------------------------+----------+
|[4.6,0.52,0.15,2.1,0.054,8.0,65.0,0.9934,3.9,0.56,1.0]  |4      |[0.0,0.0,0.0,3.0,4.0,31.0,11.0,1.0,0.0] |[0.0,0.0,0.0,0.06,0.08,0.62,0.22,0.02,0.0]                                                                            |5.0       |
|[4.7,0.6,0.17,2.3,0.058,17.0,106.0,0.9932,3.85,

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorDT = MulticlassClassificationEvaluator(labelCol="quality")
area_under_curve = evaluatorDT.evaluate(predictions)

print(area_under_curve)

0.6096614519053557


## Test predictions

In [ ]:
# On Test data - Transform test data using all the transformers and estimators in the same order 

test_df = alcohol_indexer.transform(test_df)
test_df = vector_assembler.transform(test_df)
test_predictions = dt_model.transform(test_df)

test_predictions.show(3, truncate=False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------+---------------------------------------------------------+----------------------------------------+---------------------------------------------------------------------------------------------------+----------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|pH  |sulphates|alcohol|quality|alcoholIndex|features                                                 |rawPrediction                           |probability                                                                                        |prediction|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------+---------------------------------------------------------+----------------------------------

In [ ]:
area_under_curve = evaluatorDT.evaluate(test_predictions)
area_under_curve

0.5816548284862043